CAN I SAVE THE CHANGES???

In [ ]:
!pip install uniparser_morph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/DionysiusSidorius/project-753.git

Cloning into 'project-753'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 95 (delta 33), reused 31 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 7.97 MiB | 3.57 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
from uniparser_morph import Analyzer
a = Analyzer()
# a.lexFile = "/content/project-753/lexemes.txt"
a.paradigmFile = "/content/project-753/paradigms.txt"
a.load_grammar()

fatal: destination path 'project-753' already exists and is not an empty directory.


ModuleNotFoundError: No module named 'uniparser_morph'

In [ ]:
analyses = a.analyze_words(['test', "buenos"])
print(analyses)

[[<Wordform object>
test
test; NOUN,sg
test
STEM
trans_en	testus
], [<Wordform object>
buenos
bueno; ADJ,M.Pl
buen-os
STEM
trans_en	good
]]


In [ ]:
with open('lexemes.txt', 'w') as lex:
  lex.write(
      """-lexeme
 lex: test
 stem: test.
 gramm: NOUN
 paradigm: NOM
 trans_en: testus
      """
  )
with open('paradigms.txt', 'w') as par:
  par.write(
      """-paradigm: NOM
 -flex: .
  gramm: sg
 -flex: .s
  gramm: pl
  gloss: PL
 -flex: .'s
  gramm: sg,poss
  gloss: POSS
 -flex: .s'
  gramm: pl,poss
  gloss: POSS.PL
      """
  )


In [2]:
import pandas as pd
df = pd.read_csv('/content/project-753/spanish.csv', sep=None)

<ipython-input-2-d79e48a46742>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/project-753/spanish.csv', sep=None)


In [ ]:
df.head()

,IDX,WORD,POS,MEANINGS,EXAMPLES,ADDITIONAL_INFO
0,e4348,a,sust.,"{'1': 'Letra del abecedario.', '2': 'Usado par...","{'1': ['LAP (1250?-1279?) fol. 113v87, CCN: 00...","{'Formas atestiguadas:\xa0': ['a'], 'Véase tam..."
1,e4349,a,prep.,{'1': 'Denota el complemento de la acción del ...,"{'1': ['LAP (1250?-1279?) fol. 1r33, CCN: 0000...","{'Formas atestiguadas:\xa0': ['a', 'ad', 'ha']}"
2,e132,aabruz,extranj.,{'1': 'Calcedonia. '},"{'1': ['LAP (1250?-1279?) fol. 118r41, CCN: 00...","{'Formas atestiguadas:\xa0': ['aabruz', 'aambr..."
3,e137,aanca,sust.,"{'1': 'Pieza en el juego de ajedrez indio.', '...","{'1': ['ACE (1283) fol. 82r5, CCN: 0000028, el...",{'Formas atestiguadas:\xa0': ['aanca']}
4,e138,aaptamiento,sust.,"{'1': 'Aptitud, cualidad que hace que un objet...","{'1': ['GE1 (1272-1275) fol. 169v10, CCN: 0000...",{'Formas atestiguadas:\xa0': ['a abtamiento']}


In [4]:
with open('lexemes.txt', 'w') as lex:
  pass

In [5]:
#NOUN(_V)
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+[aoeiuáóéíú]\b")) & (df['POS'].str.strip() == 'sust.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM
"""
    )

In [ ]:
#NOUN(_C)
import re
dict_noun_C = df[
    (df['WORD'].str.strip().str.fullmatch(r'.*(?!.[aoeiuáóéíú]|os).{2}')) & (df['POS'].str.strip() == 'sust.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_C['WORD'], dict_noun_C['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)es', r'\1', form) if word[-2:]!='es' else re.sub(r'(.+ess?)es', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM_C
"""
    )

In [3]:
#NOUN(_os)
import re
dict_noun_os = df[
    (df['WORD'].str.strip().str.fullmatch(r'.+os\b')) & (df['POS'].str.strip() == 'sust.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_os['WORD'], dict_noun_os['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    if any([form[-2:]=='es' for form in forms]):
      forms = [re.sub(r'(.+)es', r'\1', form) for form in forms]
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM_C
""")
    else:
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM_os
"""
    )

In [ ]:
print(dict_noun_C)

         IDX             WORD     POS  \
1336   e3759       astillejos  sust.    
7417  e20137    paralipómenos  sust.    
8060  e21875           psilos  sust.    
8061  e21876        psintamos  sust.    
8130  e22113  quebrantahuesos  sust.    
8464  e22990   reñichamientos  sust.    
9656  e26327              tos  sust.    
9909  e27596          utilios  sust.    

                                               MEANINGS  \
1336  {'1': 'Cástor y Pólux, estrellas principales d...   
7417  {'1': 'Dos libros canónicos del Antiguo Testam...   
8060               {'1': 'Tribu del norte de África. '}   
8061                           {'1': 'Cierto pájaro. '}   
8130  {'1': 'Ave del orden de las rapaces (Gypaetus ...   
8464               {'1': 'Relincho, voz del caballo. '}   
9656  {'1': 'Movimiento convulsivo y ruidoso del apa...   
9909         {'1': 'Cierta gente o nación germánica. '}   

                                               EXAMPLES  \
1336  {'1': ['GE1 (1272-1275) fol. 128r